In [1]:
import pandas as pd
import xlsxwriter

In [2]:
df = pd.read_excel("punto_de_pedido.xlsx")

In [3]:
def column_with_desired_data(raw_dataframe, art, descrip, vpd, stock ,a='',b='',c = '',d=''):

    result_df = raw_dataframe[[art,descrip,vpd,stock]]
    if a != '':
        result_df = result_df.rename(columns={art:a})
  
    if  b != '':
        result_df = result_df.rename(columns={descrip:b})

    if c != '':
        result_df = result_df.rename(columns={vpd:c})

    if d != '':
        result_df = result_df.rename(columns={stock:d})    
         
    return result_df

In [4]:
def sorted_and_cumulative_df(df, column_to_sort, cum_name):
    df = df.sort_values(column_to_sort, ascending=False)
    df[cum_name] = df[column_to_sort].cumsum()
    return df   

In [5]:
def vpd_weight(df, cum_column, weights_name):
    total_vpd = list(df[cum_column])[-1]
    df[weights_name] = df[cum_column].apply(lambda x: (x/total_vpd)*100)
    return df

In [6]:
def compare_with_limits(x,a,b):
    if x <= a:
        return 'A'
    elif  x <= b and x > a:
        return 'B'
    else:
        return 'C'

In [7]:
def abc_classification(df,weights_column,a=80, b=95, category_col_name = 'categoria'):
    df[category_col_name] = df[weights_column].apply(lambda x: compare_with_limits(x,a,b))
    return df

In [8]:
def dataframe_dictionary(complete_dataframe, category_column):
    dict_df = {}
    dict_df['A'] = complete_dataframe[complete_dataframe[category_column] == "A"]
    dict_df['B'] = complete_dataframe[complete_dataframe[category_column] == "B"]
    dict_df['C'] = complete_dataframe[complete_dataframe[category_column] == "C"]
    return dict_df

In [9]:
def stock_with_price(df1, column_df1,df2, column_df2,value_column, price_column_name,stock_column_name):
    df1 = df1.set_index(column_df1)
    df2 = df2.set_index(column_df2)
    result = pd.concat([df1, df2], axis=1, join="inner")
    result[value_column] = result[price_column_name] * result[stock_column_name]
    result[value_column] = result[value_column].apply(lambda x: int(x))
    return result

In [10]:
def delete_rows_from_df(df,article_column,*args):
    for i in args:
        df = df[df[article_column] != i]
    return df

In [11]:
def abc_statistics(complete_dataframe, classification_column,value_column,a=80,b=95):
    total_value = complete_dataframe[value_column].sum()
    lista_df=[]

    category_list=['A','B','C']
    col_headers = ['Rotación','Valorizado','Peso porcentual(%)','Target peso porcentual(%)']

    for i in category_list:
            aux_list=[]
            aux_list.append(i)
            aux_list.append(complete_dataframe[complete_dataframe[classification_column] == i][value_column].sum())
            aux_list.append(round(complete_dataframe[complete_dataframe[classification_column] == i][value_column].sum()/total_value*100,2))
            if(i == 'A'):
                aux_list.append(a)
            elif(i == 'B'):
                aux_list.append(b-a)
            else:
                aux_list.append(100-b)
            lista_df.append(aux_list)

    df_stats = pd.DataFrame(lista_df)
    df_stats = df_stats.rename(columns={0:'Categoría',1:'Valorizado',2:'Peso porcentual(%)',3:'Target peso porcentual(%)'})
    return df_stats


In [12]:
def export_excel(df_dict, df_stats, file_name):
    writer = pd.ExcelWriter(file_name + ".xlsx", engine="xlsxwriter")


    for i in df_dict:
        df_dict[i].to_excel(writer, sheet_name=i)
        workbook = writer.book
        worksheet = writer.sheets[i]
        cell_format = workbook.add_format({"border": 1})

        for j in range(140):
            worksheet.set_row(j, 25, cell_format)
            worksheet.set_column(j, width=10, last_col=12)
            if j == 2:
                worksheet.set_column(j, width=42, last_col=12)
    df_stats.to_excel(writer, sheet_name='Stats')
    workbook = writer.book
    worksheet = writer.sheets[i]
    cell_format = workbook.add_format({"border": 1})
    writer.close()

In [13]:
import classificationABC as cls

In [14]:
df = cls.column_with_desired_data(df,'ARTÍCULO','DESCRIPCIÓN','VTA. PROMEDIO','STOCK','articulo','descripcion','venta promedio','stock')

c:\Users\TERMINAL\Desktop\Development\clasificador_abc\classificationABC.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['dias de piso'] = result_df[stock] / result_df[vpd]
c:\Users\TERMINAL\Desktop\Development\clasificador_abc\classificationABC.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['dias de piso'] = result_df['dias de piso'].apply(lambda x: calculate_days_of_stock(x))


In [16]:
df = delete_rows_from_df(df,'articulo',2731,2776,2705)

In [17]:
df = sorted_and_cumulative_df(df,'venta promedio','acumulado VPD')

In [18]:
df = vpd_weight(df,'acumulado VPD','Peso de VPD acum.')

In [19]:
result_df = abc_classification(df,'Peso de VPD acum.')

In [20]:
df_pc_raw = pd.read_excel('20230518130636_PRV-1_FECVIG-20230517_plantillaPreciosCompra.xlsx', header = 1)

In [21]:
df_pc_filtered = df_pc_raw[['Artículo','Precio Neto + I. Internos']]


In [22]:
stock_classified_with_value = stock_with_price(result_df,'articulo',df_pc_filtered,'Artículo','Valorizado','Precio Neto + I. Internos','stock')

In [23]:
stock_classified_with_value = stock_classified_with_value.drop(columns=['acumulado VPD','Peso de VPD acum.'])

In [35]:
mask_con_stock = (stock_classified_with_value['stock'] != 0) & (stock_classified_with_value['venta promedio'] > 0) 

In [26]:
df_dict = dataframe_dictionary(stock_classified_with_value,'categoria')

In [27]:
df_stats = abc_statistics(stock_classified_with_value,'categoria','Valorizado')

In [28]:
cls.export_excel(df_dict, df_stats, 'Planilla procesada ABC')

In [29]:
len(df_dict['A'])

43

In [40]:
import main_process as mp

In [42]:
mp.read_instructions("instructions.txt")

This is the basic/n
model of the instruction. 



In [43]:
def read_instructions(file_path): 
    with open(file_path, "r") as file:
        file_contents = file.read()
    return file_contents  

In [50]:
inst = read_instructions('instructions.txt')

In [51]:
type(inst)

str

In [52]:
inst

'This is the basic\nmodel of the instruction. \n'